In [ ]:
import numpy as np
from ScheduleManager import ScheduleManager
from PMF import PMF
from Event import Event
from Employee import Employee
from Room import Room
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

import math
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from ipywidgets import interact
import seaborn as sns
from datetime import datetime
import time
import os
import shutil
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp
import json
from collections import Counter
from PIL import Image

In [ ]:
def Schedule(filePath, n):
    file_name = os.listdir(filePath)
    # Drop the "Opt" files and select "office_Num .csv" files
    file_name = [x for x in file_name if not "Opt" in x]
    schedule_data = []
    schedule_4_data = []

    for j in range(1, n+1):
        i = 'office_Num' + str(j) + '.csv'
        day = int(i.split('Num')[-1].split('.')[0])
        path = filePath + i

        # Read Data
        df = pd.read_csv(path)
        df = df.rename(columns = {'Office': 'Office 000', 'Office.1': 'Office 001', 'Office.2': 'Office 002', 'Office.3': 'Office 003', 'Office.4': 'Office 004',
                                  'Office.5': 'Office 005', 'Office.6': 'Office 006', 'Office.7': 'Office 007', 'Office.8': 'Office 008', 'Office.9': 'Office 009',
                                  'Office.10': 'Office 010', 'Office.11': 'Office 011', 'Office.12': 'Office 012', 'Office.13': 'Office 013', 'Office.14': 'Office 014',
                                  'Meeting room': 'Meeting Room 100', 'Meeting room.1': 'Meeting Room 101', 'Meeting room.2': 'Meeting Room 102'})
        time_interval = df[:-2][df.columns[0]]
        # Select columns meeting rooms with Occupancy data form
        df_meeting_rooms_occupancy = df[:-2][df.columns[[0, 16, 17, 18]]]
        # Select columns meeting rooms with Occupied data form
        df_meeting_rooms_occupied = df[:-2][df.columns[[16, 17, 18]]]
        df_meeting_rooms_occupied[df_meeting_rooms_occupied != 0] = 1 # replace all the occupancy data with 1
        df_meeting_rooms_occupied = pd.concat([time_interval, df_meeting_rooms_occupied], axis = 1)

        for k in range(1,4):
            # Select Meeting Room one by one
            o1 = df_meeting_rooms_occupancy[df_meeting_rooms_occupancy.columns[k]]
            o2 = df_meeting_rooms_occupied[df_meeting_rooms_occupied.columns[k]]
            # Select rows' indices which occupancy is not 0
            occupancy_indices = o1[o1 != 0].index
            occupied_indices = o2[o2 != 0].index
            # Calculate the difference of each non-zero rows' indices
            diff1 = np.diff(occupancy_indices)
            diff1 = np.append(diff1, 0)
            diff2 = np.diff(occupied_indices)
            diff2 = np.append(diff2, 0)
            # Select all the rows whose occupancy are not 0 with the indices
            data1 = o1[occupancy_indices]

            # Record the meeting information in the schedule
            count1 = 0
            for m in range(1, len(occupancy_indices) + 1):
                occupancy = data1.iloc[m - 1]
                if diff1[m - 1] == 1 and np.diff(data1)[m - 1] == 0:
                    count1 += 1
                else:
                    if k == 1:
                        schedule_data.append(['Meeting room 100', 15 * (count1 + 1), occupancy, j])
                    elif k == 2:
                        schedule_data.append(['Meeting room 101', 15 * (count1 + 1), occupancy, j])
                    else:
                        schedule_data.append(['Meeting room 102', 15 * (count1 + 1), occupancy, j])

                    count1 = 0

            # Record the meeting information in the schedule_4
            count2 = 0
            for n in range(1, len(occupied_indices) + 1):
                if diff2[n - 1] == 1:
                    count2 += 1
                else:
                    if k == 1:
                        schedule_4_data.append(['Meeting room 100', 15 * (count2 + 1), j])
                    elif k == 2:
                        schedule_4_data.append(['Meeting room 101', 15 * (count2 + 1), j])
                    else:
                        schedule_4_data.append(['Meeting room 102', 15 * (count2 + 1), j])

                    count2 = 0


    schedule = pd.DataFrame(schedule_data, columns = ['Room', 'Duration', 'Occupancy', 'Day'])
    schedule_4 = pd.DataFrame(schedule_4_data, columns = ['Room', 'Duration', 'Day'])
    
    schedule = schedule.sort_values(['Day', 'Room'])
    schedule = schedule.reset_index(drop = True)

    schedule_4 = schedule_4.sort_values(['Day', 'Room'])
    schedule_4 = schedule_4.reset_index(drop = True)


    return schedule, schedule_4

In [ ]:
# Type III: Occupancy Data PMFs
def pmf(Schedule, Meeting_Room = None, method = "Number_of_Meetings", Days = None):
    """
    Returns the pmfs of different Meeting Rooms with different methods.

    Parameters:
    Schedule (DataFrame)
    Meeting_room (string): The target meeting room's label, it should be in the schedule['Room'].
    method (string): The type of pmf to be applied.
        Must be one of 'Number_of_Meetings', 'Number_of_People', or 'Duration'. Defaults to 'Number_of_Meetings'.
    Days (integer): The target number of days to be estimated for the pmf.

    Returns:
    A probability sheet of different numbers in different method, with each number's counts and probabilities.

    Raises:
    ValueError: If the method is not one of 'Number_of_Meetings', 'Number_of_People', or 'Duration'.
    ValueError: If the Meeting_room is not in the Schedule['Room'].
    ValueError: If the Days is larger than the maximum of Schedule['Day'].

    """

    # Raise TypeError:
    if method not in ['Number_of_Meetings', 'Number_of_People', 'Duration']:
        raise TypeError("The parameter method should be one of 'Number_of_Meetings', 'Number_of_People', or 'Duration'.")
    if Meeting_Room not in Schedule['Room'].unique():
        raise TypeError("The parameter Meeting_Room should be in the meeting room list.")
    if Days > Schedule.loc[:,'Day'].max():
        raise TypeError("The parameter Days should be less or equal to the maximum of Schedule['Day']")


    # Initialization
    Meeting_room = Schedule[(Schedule['Room'] == Meeting_Room) & (Schedule['Day'] <= Days)]

    Num_Meeting = np.zeros(Days)
    Num_People = Meeting_room['Occupancy']
    Duration = Meeting_room['Duration']

    convert_dict = {'Counts': int}

    # Method "Number_of_Meetings"
    if method == 'Number_of_Meetings':
        for i in range(Days):
            Num_Meeting[i] = len(Meeting_room.loc[Meeting_room['Day'] == i+1])

        ar,num = np.unique(Num_Meeting, return_counts = True) # Identify the different numbers in Num_Meeting and calculater their counts.
        Prob_Num_Meeting = np.zeros(len(ar))
        for j in range(len(ar)):
            Prob_Num_Meeting[j] = num[j] / sum(num) # Calculate the probabilities of different numbers in Num_Meeting

        a = pd.DataFrame([ar, num, Prob_Num_Meeting]).T
        a.columns = ['Number of Meetings', 'Counts', 'Probability']
        x = pd.DataFrame(np.arange(0,10,1))
        x.columns = ['Number of Meetings']
        pmf = pd.merge(x,a, how = 'outer', on = ['Number of Meetings'])



    # Method "Number_of_People"
    if method == 'Number_of_People':
        ar,num = np.unique(Num_People, return_counts = True) # Identify the different numbers in Num_People and calculater their counts.
        Prob_Num_People = np.zeros(len(ar))
        for i in range(len(ar)):
            Prob_Num_People[i] = num[i] / sum(num) # Calculate the probabilities of different numbers in Num_People

        a = pd.DataFrame([ar, num, Prob_Num_People]).T
        a.columns = ['Number of People', 'Counts', 'Probability']
        x = pd.DataFrame(np.arange(0,10,1))
        x.columns = ['Number of People']
        pmf = pd.merge(x,a, how = 'outer', on = ['Number of People'])


    # Method "Duration"
    if method == 'Duration':
        ar,num = np.unique(Duration, return_counts = True) # Identify the different numbers in Duration and calculater their counts.
        Prob_Duration = np.zeros(len(ar))
        for i in range(len(ar)):
            Prob_Duration[i] = num[i] / sum(num) # Calculate the probabilities of different numbers in Duration

        a = pd.DataFrame([ar, num, Prob_Duration]).T
        a.columns = ['Duration(Mins)', 'Counts', 'Probability']
        x = pd.DataFrame(np.arange(30,301,30))
        x.columns = ['Duration(Mins)']
        pmf = pd.merge(x,a, how = 'outer', on = ['Duration(Mins)'])


    pmf = pmf.fillna(0)
    pmf = pmf.astype(convert_dict) # Transfer column 'Counts' from float to integer

    return pmf

# Type IV: Occupied Data PMFs
def pmf_4(Schedule, Meeting_Room = None, method = "Number_of_Meetings", Days = None):
    """
    Returns the pmfs of different Meeting Rooms with different methods.

    Parameters:
    Schedule (DataFrame)
    Meeting_room (string): The target meeting room's label, it should be in the schedule['Room'].
    method (string): The type of pmf to be applied.
        Must be one of 'Number_of_Meetings', 'Number_of_People', or 'Duration'. Defaults to 'Number_of_Meetings'.
    Days (integer): The target number of days to be estimated for the pmf.

    Returns:
    A probability sheet of different numbers in different method, with each number's counts and probabilities.

    Raises:
    ValueError: If the method is not one of 'Number_of_Meetings', 'Number_of_People', or 'Duration'.
    ValueError: If the Meeting_room is not in the Schedule['Room'].
    ValueError: If the Days is larger than the maximum of Schedule['Day'].

    """

    # Raise TypeError:
    if method not in ['Number_of_Meetings', 'Duration']:
        raise TypeError("The parameter method should be one of 'Number_of_Meetings' or 'Duration'.")
    if Meeting_Room not in Schedule['Room'].unique():
        raise TypeError("The parameter Meeting_Room should be in the meeting room list.")
    if Days > Schedule.loc[:,'Day'].max():
        raise TypeError("The parameter Days should be less or equal to the maximum of Schedule['Day']")


    # Initialization
    Meeting_room = Schedule[(Schedule['Room'] == Meeting_Room) & (Schedule['Day'] <= Days)]

    Num_Meeting = np.zeros(Days)
    Duration = Meeting_room['Duration']

    convert_dict = {'Counts': int}

    # Method "Number_of_Meetings"
    if method == 'Number_of_Meetings':
        for i in range(Days):
            Num_Meeting[i] = len(Meeting_room.loc[Meeting_room['Day'] == i+1])

        ar,num = np.unique(Num_Meeting, return_counts = True) # Identify the different numbers in Num_Meeting and calculater their counts.
        Prob_Num_Meeting = np.zeros(len(ar))
        for j in range(len(ar)):
            Prob_Num_Meeting[j] = num[j] / sum(num) # Calculate the probabilities of different numbers in Num_Meeting

        a = pd.DataFrame([ar, num, Prob_Num_Meeting]).T
        a.columns = ['Number of Meetings', 'Counts', 'Probability']
        x = pd.DataFrame(np.arange(0,10,1))
        x.columns = ['Number of Meetings']
        pmf = pd.merge(x,a, how = 'outer', on = ['Number of Meetings'])



    # Method "Duration"
    if method == 'Duration':
        ar,num = np.unique(Duration, return_counts = True) # Identify the different numbers in Duration and calculater their counts.
        Prob_Duration = np.zeros(len(ar))
        for i in range(len(ar)):
            Prob_Duration[i] = num[i] / sum(num) # Calculate the probabilities of different numbers in Duration

        a = pd.DataFrame([ar, num, Prob_Duration]).T
        a.columns = ['Duration(Mins)', 'Counts', 'Probability']
        x = pd.DataFrame(np.arange(30,301,30))
        x.columns = ['Duration(Mins)']
        pmf = pd.merge(x,a, how = 'outer', on = ['Duration(Mins)'])


    pmf = pmf.fillna(0)
    pmf = pmf.astype(convert_dict) # Transfer column 'Counts' from float to integer

    return pmf

In [ ]:
PMF_mode = 1

if PMF_mode == 1:
    meeting_durations_PMF = PMF([30, 60, 90, 120], [0.2, 0.6, 0.1, 0.1])
    number_of_employees_PMF = PMF([2, 3, 4, 5], [0.5, 0.15, 0.05, 0.3])
    number_of_meetings_PMF = PMF([2, 3, 4, 5], [0.25, 0.25, 0.25, 0.25])
elif PMF_mode == 2:
    meeting_durations_PMF = PMF([30, 60, 90, 120, 150, 180], [0.05, 0.62, 0.06, 0.14, 0.05, 0.08])
    number_of_employees_PMF = PMF([2, 3, 4, 5], [0.64, 0.04, 0.11, 0.21])
    number_of_meetings_PMF = PMF([0, 1, 2, 3, 4, 5, 6, 7], [0.02, 0.03, 0.27, 0.36, 0.24, 0.03, 0.02])
else:
    print("Please provide the correct PMF.")


# Create the room list
number_of_offices = 15
number_of_meeting_rooms = 3

print("Number of offices: " + str(number_of_offices))
print("Number of meeting rooms: " + str(number_of_meeting_rooms))

# Room use schedule - Meetings will only be scheduled during these times
# Beginning
start_time = datetime(2010, 1, 1, 8, 0, 0)
end_time = datetime(2010, 1, 1, 17, 0, 0)
event_type = "Room is available for use"
room_use = Event(start_time, end_time, event_type, None, None)

# Shift 1
# Morning
start_time = datetime(2010, 1, 1, 8, 0, 0)
end_time = datetime(2010, 1, 1, 12, 0, 0)
event_type = "Morning"
e1 = Event(start_time, end_time, event_type, None, None)

# Afternoon
start_time = datetime(2010, 1, 1, 13, 0, 0)
end_time = datetime(2010, 1, 1, 17, 0, 0)
event_type = "Afternoon"
e2 = Event(start_time, end_time, event_type, None, None)

# Create the employee list
number_of_employees = number_of_offices
print("Number of employees: " + str(number_of_employees))


In [ ]:
# Define the simulation days needed
sample = np.array([10])

# Defined the experiment times needed
experiment = 100

In [ ]:
for y in range(1, experiment+1): # Experiment Times
    for x in range(1, len(sample) + 1): # Days Data
        sample_size = sample[x-1]
        
        for i in range(1,sample_size + 1):
            office_room_list = []
            # Offices
            for office_number in range(number_of_offices):
                office_room_list.extend([Room("Office", "00" + str(office_number), 2.34520787991, 2, 0.1, 0, 1,
                                                meeting_durations_PMF, number_of_employees_PMF, number_of_meetings_PMF)])
                office_room_list[office_number].working_schedule.add_event(room_use)

            meeting_rooms_list = []
            
            # Meeting rooms
            for meeting_room_number in range(number_of_meeting_rooms):
                                        meeting_rooms_list.extend([Room("Meeting room", "10" + str(meeting_room_number), 2.34520787991 * 5, 2, 0.1, 5,
                                                                        0, meeting_durations_PMF, number_of_employees_PMF, number_of_meetings_PMF)])
                
                                        meeting_rooms_list[meeting_room_number].working_schedule.add_event(room_use)

            # Employees
            employees_list = []
            for employee_number in range(number_of_employees):
                employees_list.extend([Employee("000" + str(employee_number), "Worker", office_room_list[employee_number])])

            # Add the schedules to the employees
            for employee_index in range(15):
                employees_list[employee_index].add_work_event(e1)
                employees_list[employee_index].add_work_event(e2)

            # Initialise the schedule manager
            my_scheduler = ScheduleManager(office_room_list, meeting_rooms_list, employees_list)
            my_scheduler.setup(filepath + 'Experiment_' + str(y) + '/Opt_office_Num' + str(i) + '.csv',
                               filepath + 'Experiment_' + str(y) + '/office_Num' + str(i) + '.csv')
            print(str(i) + 'csv file done.')

        # Type II PMFs
        source_folder = filepath
        destination_folder = filepath + 'Experiment_' + str(y) + '/' 

        file_to_move1 = "num_of_people.json"
        file_to_move2 = "duration_of_meetings_min.json"
        file_to_move3 = "num_of_meetings.json"

        source_path1 = os.path.join(source_folder, file_to_move1)
        destination_path1 = os.path.join(destination_folder, file_to_move1)

        source_path2 = os.path.join(source_folder, file_to_move2)
        destination_path2 = os.path.join(destination_folder, file_to_move2)

        source_path3 = os.path.join(source_folder, file_to_move3)
        destination_path3 = os.path.join(destination_folder, file_to_move3)

        shutil.copyfile(source_path1, destination_path1)
        shutil.copyfile(source_path2, destination_path2)
        shutil.copyfile(source_path3, destination_path3)
        
        os.remove(source_path1)
        os.remove(source_path2)
        os.remove(source_path3)
        
        


        # Type III PMFs
        schedule,schedule_4 = Schedule(destination_folder, sample_size)
        schedule.to_csv(filepath + 'Experiment_' + str(y) + '/' + 'schedule.csv', index=False)


        # Type IV PMFs
        schedule_4.to_csv(filepath + 'Experiment_' + str(y) + '/' + 'schedule_4.csv', index=False)



